## Training a Single-Output ANN from scratch

Note: this script trains only one ANN for the station selected by user. Please set 'output_stations' to different stations for training multiple ANNs.

In [1]:
######## User Settings ########
#### 1. Select parameters to be used ####
input_var = ['SAC','Exp','SJR','DICU','Vern','SF_Tide','DXC']
#### 2. Select stations to be predicted ####
# choose one of 'Emmaton', 'Jersey Point', 'Collinsville', 'Rock Slough', 'Antioch', 'Mallard',
# 'LosVaqueros', 'Martinez', 'MiddleRiver', 'Vict Intake', 'CVP Intake', 'CCFB_OldR'
output_stations=['CCFB_OldR']
#### 3. Specify directory to excel dataset and the helper script (folder name only) ####
#### if data is uploaded instead of in your google drive, set data_in_google_drive=False #####
data_in_google_drive = True
google_drive_dir = 'python_ANN'

output_shape = 1
nn_shape = [None,8*output_shape,2*output_shape,1*output_shape] # number of neurons in each layer


##1. Read data

In [2]:
from google.colab import drive
import os
# Mount Google drive
if data_in_google_drive:
    drive.mount('/content/drive',force_remount=True)
    xl_path = os.path.join('/content/drive','My Drive',google_drive_dir,"ANN_data2.xlsx")
    %tensorflow_version 1.x
else:
    xl_path = "ANN_data2.xlsx"

# import helper functions
import sys
!export PYTHONPATH=""
sys.path.append(os.path.join('/content/drive','My Drive',google_drive_dir))
from ann_helper import read_data,normalize_in,writeF90,initnw,show_eval

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
TensorFlow 1.x selected.


In [3]:
import tensorflow as tf
import math
import numpy as np
from sklearn.model_selection import train_test_split
import time
# from sklearn.utils import shuffle
from scipy import stats

test_mode = False

locs = {'Emmaton':0,'Jersey Point':1,'Collinsville':2,'Rock Slough':3,'Antioch':4,
        'Mallard':5, 'LosVaqueros':6, 'Martinez':7, 'MiddleRiver':8, 'Vict Intake':9,
        'CVP Intake':10, 'CCFB_OldR':11}
abbrev_map = {'rock slough':'ORRSL','rockslough':'ORRSL',
            'emmaton':'EMM','jersey point':'JP','jerseypoint':'JP',
            'antioch':'antioch','collinsville':'CO',
            'mallard':'Mallard','mallard island':'Mallard',
            'los vaqueros':'LosVaqueros','losvaqueros':'LosVaqueros',
            'martinez':'MTZ',
            'middle river':'MidR_intake','MiddleRiver':'MidR_intake',
            'victoria cannal':'Victoria_intake','Vict Intake':'Victoria_intake',
            'cvp intake':'CVP_intake','clfct forebay':'CCFB',
            'clfct forebay intake':'CCFB_intake','x2':'X2'};

output_stations = sorted(output_stations,key=lambda x: locs[x])

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))


on_server = True

# LM optimizer settings
max_fail = 3
epochs = 10
init_mu = .05
mu_max = 1e10
target_mse = 0.

# adam optimizer settings
adam_epochs = 100
batch_size = 32


if test_mode or device_name != '/device:GPU:0':
    epochs = 5
else:
    epochs = 100

train_loc = locs[output_stations[0]]
if output_stations[0].lower() in abbrev_map.keys():
    ann_name = abbrev_map[output_stations[0].lower()] 
else:
    ann_name = output_stations[0].lower()

start = time.time()

# read data from excel
x_data,y_data = read_data(xl_path,input_var,output_stations)
end = time.time()
print("loading data in %.2f seconds" % (end-start) )
    
# normalize data to 0.1 ~ 0.9
[x_norm,x_slope,x_bias] = normalize_in(x_data)
[y_norm,y_slope,y_bias] = normalize_in(y_data)

# split 80% for training, 20% for testing
x_train_ori, x_test_ori, y_train0, y_test0 = train_test_split(x_norm,
                                                              y_norm,
                                                              test_size=0.2,
                                                              random_state = 0)

if test_mode:
    x_train_ori = x_train_ori[:100]
    x_test_ori = x_test_ori[:100]
    y_train0 = y_train0[:100]
    y_test0 = y_test0[:100]
train_err = []
test_err = []
train_shape = len(x_train_ori)
nn_shape[0] = x_train_ori.shape[1]

Found GPU at: /device:GPU:0
Disgarding last 1 row(s) of data in output set...
loading data in 43.95 seconds
loading data in 43.99 seconds


In [5]:
# build artificial neural network
tf.compat.v1.reset_default_graph()
tf.compat.v1.set_random_seed(1)

x = tf.compat.v1.placeholder(tf.float32, [None, nn_shape[0]], name='InputData')
y = tf.compat.v1.placeholder(tf.float32, [None, 1], name='LabelData')

init_val = initnw(list(zip(*(nn_shape[i:] for i in range(2)))),x_train_ori)

W1 = tf.Variable(initial_value=init_val[0][0], name='w1',dtype='float32')
b1 = tf.Variable(initial_value=init_val[0][1], name='b1',dtype='float32')
W2 = tf.Variable(initial_value=init_val[1][0], name='w2',dtype='float32')
b2 = tf.Variable(initial_value=init_val[1][1], name='b2',dtype='float32')
W3 = tf.Variable(initial_value=init_val[2][0], name='w3',dtype='float32')
b3 = tf.Variable(initial_value=init_val[2][1], name='b3',dtype='float32')

with tf.compat.v1.name_scope('layer1'):
    first_out = tf.sigmoid(tf.add(tf.matmul(x,W1),b1))
with tf.compat.v1.name_scope('layer2'):
    second_out = tf.sigmoid(tf.add(tf.matmul(first_out,W2),b2))
# with tf.name_scope('layer3'):
#     third_out = tf.sigmoid(tf.add(tf.matmul(second_out,W3),b3))
with tf.compat.v1.name_scope('layer3'):
    pred = tf.matmul(second_out, W3) + b3

# residuals
r = tf.subtract(pred,y)
# loss function must be mse or sse for LM
cost = tf.reduce_mean(input_tensor=tf.square(tf.subtract(pred,y)))


# For adam optimizer:
global_step = tf.Variable(0, trainable=False)
initial_lr = 1e-2
learning_rate = tf.compat.v1.train.exponential_decay(initial_lr, global_step,
                                       40, 0.999, staircase=True)
# Adam optimizer
with tf.compat.v1.name_scope('train_op'):
    adam_opt = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost,global_step = global_step)

# LM optimizer
with tf.compat.v1.name_scope('train_op'):
    opt = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=1)

with tf.compat.v1.name_scope('Accuracy'):
    # Accuracy
    acc = tf.reduce_mean(input_tensor=tf.square(tf.subtract(pred,y)))

# Initialize the variables (i.e. assign their default value)
init = tf.compat.v1.global_variables_initializer()


# LM algorithm
def jacobian(y, x):
    loop_vars = [
        tf.constant(0, tf.int32),
        tf.TensorArray(tf.float32, size=train_shape),
    ]

    _, jacobian = tf.while_loop(
        cond=lambda i, _: i < train_shape,
        body=lambda i, res: (i+1, res.write(i, tf.reshape(tf.gradients(ys=y[i], xs=x), (-1,)))),
        loop_vars=loop_vars)
    return jacobian.stack()


r_flat = tf.expand_dims(tf.reshape(r,[-1]),1)
parms = [W1, b1, W2, b2, W3, b3]
parms_sizes = [tf.size(input=p) for p in parms]
j = tf.concat([jacobian(r_flat, p) for p in parms], 1)
jT = tf.transpose(a=j)
hess_approx = tf.matmul(jT, j)
grad_approx = tf.matmul(jT, r_flat)

mu = tf.compat.v1.placeholder(tf.float32, shape=[])

store = [tf.Variable(tf.zeros(p.shape, dtype=tf.float32)) for p in parms]
save_parms = [tf.compat.v1.assign(s, p) for s, p in zip(store, parms)]
restore_parms = [tf.compat.v1.assign(p, s) for s, p in zip(store, parms)]

wb_flat = tf.concat([tf.reshape(p,[-1,1]) for p in parms],axis=0)
n = tf.add_n(parms_sizes)
I = tf.eye(n, dtype=tf.float32)
w_2 = tf.reduce_sum(input_tensor=tf.square(wb_flat))


sess = tf.compat.v1.Session()
sess.run(init)
print("Number of trainable parameters: %d" % sess.run(n))


#  lm algorithm
dp_flat = tf.matmul(tf.linalg.inv(hess_approx + tf.multiply(mu, I)), grad_approx)
dps = tf.split(dp_flat, parms_sizes, 0)

for i in range(len(dps)):
    dps[i] = tf.reshape(dps[i], parms[i].shape)
lm = opt.apply_gradients(zip(dps, parms))


Number of trainable parameters: 1037


In [7]:
saver = tf.compat.v1.train.Saver({'W1':W1,'b1':b1,'W2':W2,'b2':b2,'W3':W3,'b3':b3})

In [ ]:
# Adam optimizer training


# Feed batch data
def get_batch(inputX, inputY, batch_size):
    duration = len(inputX)
    for i in range(0,duration//batch_size):
        idx = i*batch_size
        yield inputX[idx:idx+batch_size], inputY[idx:idx+batch_size]




# Start training

start = time.time()
with sess.as_default():
    with sess.graph.as_default():
        cost_list = []
        lr_list = []
        # Run the initializer
        sess.run(init)

        # Training cycle
        total_batch = train_shape//batch_size
        for epoch in range(adam_epochs):
            avg_cost = 0.
            ii = 0
            # Loop over all batches
            for batch_xs, batch_ys in get_batch(x_train_ori, y_train0,batch_size):
                ii = ii +1
                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c = sess.run([adam_opt, cost],
                                 feed_dict={x: batch_xs, y: batch_ys})
                # Write logs at every iteration
                # Compute average loss
                avg_cost += c / total_batch
            lr = learning_rate.eval()
            cost_list.append(avg_cost)
            lr_list.append(lr)
            # Display logs per epoch step
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost),
                  "lr = ","{:.10f}".format(lr))

    print("Optimization Finished!")

    # Test model
    # Calculate accuracy
    print("Train Error:", acc.eval({x: x_train_ori, y: y_train0}))
    print("Test Error:", acc.eval({x: x_test_ori, y: y_test0}))
    y_test_ = sess.run(pred,feed_dict={x:x_test_ori})
    print("Test MSE:", np.mean(np.square((y_test_-y_test0)/y_slope)))
    print("Test MAPE:", np.mean(np.abs(y_test_-y_test0)/y_test0))


In [9]:
with sess.as_default():
    with sess.graph.as_default():
        print("Train Error before LM:", acc.eval({x: x_train_ori, y: y_train0}))

Train Error before LM: 0.0048930175


In [ ]:

feed_dict = {x: x_train_ori,
             y: y_train0}
feed_dict[mu] = init_mu

# construct so-called feed dictionary to map placeholders to actual values

validation_feed_dict = {x: x_test_ori,
                        y: y_test0}

train_break = False

current_loss = sess.run(cost,{x: x_train_ori,y: y_train0})


# Start training
start = time.time()
with sess.as_default():
    with sess.graph.as_default():
        epoch = 1
        fail_step = 0
        # Training cycle
        while epoch < epochs and current_loss > target_mse:
            if not(epoch%2) or epoch == 1:
                val_loss = sess.run(cost, validation_feed_dict)
                print('epoch: %3d , mu: %.5e, train loss: %.10f, val loss: %.10f'%(epoch,feed_dict[mu],current_loss,val_loss))
            sess.run(save_parms)
            while True:
                start_step = time.time()
                sess.run(lm, feed_dict)
                new_loss = sess.run(cost,feed_dict)
                # print('One update ended in %d seconds' % (time.time()-start_step))
                if new_loss > current_loss:
                    fail_step += 1
                    feed_dict[mu] *= 10
                    if feed_dict[mu] > mu_max or fail_step > max_fail:
                        train_break = True
                        break
                    sess.run(restore_parms)
                else:
                    print("mu: %.5e, new loss: %.5f, previous loss: %.5f"%(feed_dict[mu],new_loss,current_loss))
                    fail_step = 0
                    feed_dict[mu] /= 10
                    feed_dict[mu] = max(1e-20,feed_dict[mu])
                    current_loss = new_loss
                    break
            if train_break:
                print('Failed for %d step(s), current mu = %.5e, stop training' % (fail_step,feed_dict[mu]))
                break
            epoch += 1
    print("Location #%d Optimization Finished in %d seconds!" %(train_loc, time.time() - start))

    # Test model

    y_train_predicted = sess.run(pred,feed_dict)
    y_test_predicted = sess.run(pred,validation_feed_dict)

    show_eval(y_train_predicted[:,0],
              y_train0[:,0],
              y_test_predicted[:,0],
              y_test0[:,0],
              y_slope,y_bias,ann_name)
    writeF90(ann_name,
             y_slope[0],y_bias[0],
             sess.run(W1).transpose(),sess.run(b1),
             sess.run(W2).transpose(),sess.run(b2),
             sess.run(W3).transpose(),sess.run(b3))

model_path = "./%s/model.ckpt" % (ann_name)
save_path = saver.save(sess, model_path)
print("Model saved in path: %s" % save_path)
with sess.as_default():
    with sess.graph.as_default():
        saved_test_loss = acc.eval(validation_feed_dict)
        print("Train Error after LM:", acc.eval({x: x_train_ori, y: y_train0}))
        print("Test Error after LM:", saved_test_loss)


epoch:   1 , mu: 5.00000e-02, train loss: 0.0048930175, val loss: 0.0050581386
